# Export

#### Overview
Notebook for exporting CATMAID data to iCAT-MAID-data repository for use with `OpenSeadragon`

In [34]:
from pathlib import Path
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage import (img_as_ubyte, img_as_float,
                     io, color, util, exposure)

In [2]:
# Add modules to PATH
# -------------------
import sys
fp = Path('./notebooks/')
if fp.as_posix() not in sys.path:
    sys.path.insert(0, fp.as_posix())

# Import modules
from dataframe import compile_DataFrame
from colorize import transform, trevni

In [63]:
# Suppress warnings
import warnings
warnings.simplefilter('ignore', UserWarning)

## Organize
---

In [49]:
# Project name
# ------------
project = '20200507_RL012'

# Project location
# ----------------
project_dir = Path('//sonic/long_term_storage/rlane/CATMAID/projects/')
project_dir /= project

# Stacks to export
# ----------------
stacks = ['lil_EM_montaged',
          'hoechst_correlated',
          'insulin_correlated']

# Export directory
export_dir = Path('M://tnw/ist/do/projects/iCAT/development/iCAT-MAID-data/projects')
export_dir /= project

# Compile project DataFrame
# -------------------------
df = compile_DataFrame(project_dir,
                       stacks=stacks)

# Preview
# -------
out = f"""\
Location.... {project_dir}
Export to... {export_dir}
Project..... {project}
Stacks...... {stacks}
...
"""
print(out)
df.sample(5)

Location.... \\sonic\long_term_storage\rlane\CATMAID\projects\20200507_RL012
Export to... M:\tnw\ist\do\projects\iCAT\development\iCAT-MAID-data\projects\20200507_RL012
Project..... 20200507_RL012
Stacks...... ['lil_EM_montaged', 'hoechst_correlated', 'insulin_correlated']
...



stack,z,y,x,zoom,hoechst_correlated,insulin_correlated,lil_EM_montaged
12504,6,28,8,0,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...
5162,2,24,31,0,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...
8864,4,24,13,0,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...
1558,0,31,27,0,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...
2292,1,8,6,1,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...


## Export
---

#### Postprocessing settings

In [80]:
# Color transforms
# ----------------
color_transform = {
    'hoechst_correlated': [[0.2, 0.0, 0.0, 0.2],
                           [0.0, 0.2, 0.0, 0.2],
                           [0.0, 0.0, 1.0, 1.0],
                           [0.0, 0.0, 0.0, 0.0]],

    'insulin_correlated': [[1.0, 0.0, 0.0, 1.0],
                           [0.0, 0.6, 0.0, 0.6],
                           [0.0, 0.0, 0.0, 0.0],
                           [0.0, 0.0, 0.0, 0.0]],
}

# Transparencies
# --------------
alphas = {
    'hoechst_correlated': 1.0,
    'insulin_correlated': 0.6,
    'lil_EM_montaged':    1.0
}

#### Filter

In [81]:
# Set min/max zoom levels
# -----------------------
zoom_min = 2
zoom_max = 6

# Filter to min/max zoom levels
# -----------------------------
source = df.loc[(df['zoom'] >= zoom_min) &\
                (df['zoom'] <= zoom_max)].dropna(axis=0)

# Preview
# -------
source.sample(3)

stack,z,y,x,zoom,hoechst_correlated,insulin_correlated,lil_EM_montaged
11346,6,4,9,2,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...
95,0,3,10,2,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...
5934,3,1,2,4,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...,\\sonic\long_term_storage\rlane\CATMAID\projec...


#### Export

In [83]:
# Iterate through tile images
for i, row in tqdm(source.loc[:].iterrows(),
                   total=len(source)):

    fps = row[stacks]
    images = []
    for stack in stacks:
        if 'EM' in stack:
            im = trevni(fps[stack])
        else:
            im = transform(fps[stack], color_transform[stack])
        images.append(im * alphas[stack])
    composite = np.sum(images, axis=0)

    rescaled = exposure.rescale_intensity(composite,
                                          in_range=(0, 1))

    # Save to iCAT-MAID-data project
    tgt = export_dir / f"{row['z']:.0f}"
    tgt.mkdir(parents=False, exist_ok=True)
    tgt /= f"{row['y']:.0f}_{row['x']:.0f}_{row['zoom']:.0f}.png"
    io.imsave(tgt.as_posix(), img_as_ubyte(rescaled))